<a href="https://colab.research.google.com/github/ndhartanto/ndhartanto/blob/main/Project_Skripsi_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -qq --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
!pip install -qq nest_asyncio
!pip install goslate 

import twint
import nest_asyncio
nest_asyncio.apply()
import csv
import re
import goslate

gs = goslate.Goslate()

import pandas as pd
import nltk
from nltk.corpus import stopwords
import pandas as pd

nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# creating object of SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()


import spacy
import string
sp = spacy.load('en_core_web_sm')

# for stopwords removal
from spacy.lang.en import English
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()
from spacy.lang.en.stop_words import STOP_WORDS


     |████████████████████████████████| 1.3MB 5.0MB/s 
     |████████████████████████████████| 266kB 18.7MB/s 
     |████████████████████████████████| 358kB 21.7MB/s 
     |████████████████████████████████| 296kB 23.8MB/s 
     |████████████████████████████████| 143kB 25.2MB/s 
     |████████████████████████████████| 296kB 24.0MB/s 


In [ ]:
# HEADING FOR THE CSV 

csvFile = open('data_perdagangan.csv', 'w')

csvWriter = csv.writer(csvFile)
csvWriter.writerow(["Date & Time", "Username", "Tweet"])
csvFile.close()

In [ ]:
# CONTENT 

keywords = ["perdagangan indonesia china",
            "perdagangan indonesia cina",]

for x in keywords :

  c = twint.Config()
  c.Search = x 
  c.Limit = 2000
  c.Since = "2020-03-30"
  c.Until = "2021-03-30"
  c.Store_object = True
  c.Hide_output = True
  c.Filter_retweets = True
  c.Lang = "ID"

  twint.run.Search(c)
  tweets = twint.output.tweets_list

def remove_duplicates(tweets):
  newTweets = []
  ids = []

  for tweet in tweets:
    if (tweet.id not in ids):
      ids.append(tweet.id)
      newTweets.append(tweet)
    return newTweets

tweets = remove_duplicates(tweets)

csvFile = open('data_perdagangan.csv', 'a')

csvWriter = csv.writer(csvFile)

for tweet in tweets:
  csvWriter.writerow([tweet.datetime, tweet.username, tweet.tweet])
  print(tweet.datetime, tweet.username, tweet.tweet)
csvFile.close()

In [ ]:
# CLEANING 

csvFile = open('data_perdagangan.csv', 'r', encoding="utf8", errors='ignore')
newFile = open('data_clean.csv', 'w')
rows = csv.reader(csvFile)
csvWriter = csv.writer(newFile)

def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
     SOURCE : https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/calculate-tweet-word-frequencies-in-python/
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", txt).split())

def remove_usernames(txt):
    """ removing twitter usernames / mentions 
    (example : "Hello @user123 how are you" becomes "Hello how are you")
    SOURCE : https://stackoverflow.com/questions/50830214/remove-usernames-from-twitter-data-using-python"""

    return re.sub('@[\w]+','',txt)

def remove_hashtags(txt):
    """removing hashtags
    (example : "Hello #howareyou" becomes "Hello")"""
    return re.sub('#[\w]+','',txt)

def remove_numbers(txt):
    """removing numbers
    (example : "Hello 12 how are you" becomes "Hello how are you")"""
    return re.sub(r'[0-9\.]+', '', txt)

def remove_entity(txt):
    """removing special HTML entity such as &gt; , &amp; , etc"""
    return re.sub('&[\w]+;', '', txt)

for row in rows:
  tweet = row[2] 
  tweet = remove_usernames(tweet)
  tweet = remove_hashtags(tweet)
  tweet = remove_entity(tweet)
  tweet = remove_url(tweet)
  tweet = remove_numbers(tweet)

  print(tweet)
  row.append(tweet)
  csvWriter.writerow(row)


In [ ]:
# TRANSLATING

csvFile = open('data_clean.csv', 'r') #, encoding="utf8", errors='ignore')
newFile = open('data_trans.csv', 'w')
rows = csv.reader(csvFile)
csvWriter = csv.writer(newFile)

for row in rows:
  sentence = row[3] 
  sentence = gs.translate(sentence, 'en')
  sentence.lower()
  print(sentence)
  row.append(sentence)
  csvWriter.writerow(row)

In [ ]:
# VADER POLARITY 

# loading csv file.
df = pd.read_csv('tweet_eng_only.csv', encoding='latin-1')

df.head()

# Checking
# sid.polarity_scores(df.iloc[0]['text'])

df['scores'] = df['text'].apply(lambda review:sid.polarity_scores(review))
df['compound'] = df['scores'].apply(lambda d:d['compound'])
df['label'] = df['compound'].apply(lambda score: 'POSITIF' if score >0 else 'NEGATIF' if score <0 else 'NETRAL')
df.head()

# df.to_csv('labeled_vader.csv', index=False)


In [ ]:
# LEMMATIZATION & STOPWORDS REMOVAL

csvFile = open('labeled_vader.csv', 'r')
newFile = open('spacy_vader_LEMMA.csv', 'w')
rows = csv.reader(csvFile)
csvWriter = csv.writer(newFile)

for row in rows:
  # lower case
  sentence = row[0].lower()

  # remove punctuation if any
  sentence = sentence.translate(str.maketrans('', '', string.punctuation))

  sentence = sp(sentence)
    
  # LEMMATISATION
  lemma_list = []
  for word in sentence:
     # CHECK
     # print(word.text, word.lemma_)
     lemma_list.append(word.lemma_)

  no_stopwords =[] 

  for word in lemma_list:
    lexeme = nlp.vocab[word]
    if lexeme.is_stop == False:
      no_stopwords.append(word) 
  print(lemma_list)
  print(no_stopwords)   

  row.append(lemma_list)
  row.append(no_stopwords)
  csvWriter.writerow(row)

df = pd.read_csv('spacy_vader_LEMMA.csv') #,sep='\t')
df_new = df.rename(columns={"['text'].1" : 'text_final'})
df_new.head() #CHECK


In [ ]:
# CONFUSION MATRIX - Accuracy, Precision, Recall, Fscore

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.metrics import precision_recall_fscore_support as score

# HOLD-OUT - Dividing testing and training data
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df_new['text_final'],df_new['label'],test_size=0.3)

# Label encode - transform Categorical data of string (negatif, netral, positif) into numerical values (0,1,2)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

# TF-IDF Vectorizer 
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df_new['text_final'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

# CHECKING
# print(Tfidf_vect.vocabulary_)
# print(Train_X_Tfidf)

# NAIVE BAYES & SVM

# Classifier - Algorithm - Naive Bayes

# fit the training dataset on the classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)

# Classifier - Algorithm - SVM

# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

conf_matrix = confusion_matrix(Test_Y, predictions_SVM)
class_label = ["NEGATIF", "NETRAL", "POSITIF"]
test = pd.DataFrame(conf_matrix, index = class_label, columns = class_label)
sns.heatmap(test, annot = True, fmt = "d")
plt.title("Confusion Matrix SVM")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# SVM RESULT
print("SVM RESULT")
print("Accuracy Score =",accuracy_score(predictions_SVM, Test_Y))
precision, recall, fscore, support = score(Test_Y, predictions_SVM, average='weighted')
print("Precision =", precision)
print("Recall =", recall)
print("F-score =", fscore)

conf_matrix = confusion_matrix(Test_Y, predictions_NB)
class_label = ["NEGATIF", "NETRAL", "POSITIF"]
test = pd.DataFrame(conf_matrix, index = class_label, columns = class_label)
sns.heatmap(test, annot = True, fmt = "d")
plt.title("Confusion Matrix Naive Bayes")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# NAIVE BAYES RESULT
print("NAIVE BAYES RESULT")
print("Accuracy Score =",accuracy_score(predictions_NB, Test_Y))
precision, recall, fscore, support = score(Test_Y, predictions_NB, average='weighted')
print("Precision =", precision)
print("Recall =", recall)
print("F-score =", fscore)
